# Notificación de Falla Crítica por Calidad de Datos

Propósito: Construir y enviar una alerta por correo cuando una regla de calidad crítica falla y bloquea la ejecución del pipeline.

Rol: capa de control/alertas dentro del flujo Bronze en Databricks — comunica al equipo responsable los detalles de la falla para acción inmediata.

In [0]:
# NOTIFICACIÓN POR CORREO
import json
import smtplib
from datetime import datetime
from email.message import EmailMessage
from io import StringIO

import pandas as pd


In [0]:
# 1. DEFINIR PARÁMETROS (WIDGETS)
dbutils.widgets.text("send_to", "xxxxx@gmail.com", "Enviar al Correo")
dbutils.widgets.text("lista_detalle", "", "Lista de calidad")

In [0]:
# 2. OBTENER PARÁMETROS
send_to = dbutils.widgets.get("send_to")
mensaje = dbutils.widgets.get("lista_detalle")

In [0]:
# 3. CONFIGURACIÓN DE SMTP
sender_email = "xxxx@gmail.com"
clave_app = "xxxx"
smtp_server = "smtp.gmail.com"
smtp_port = 587

In [0]:
# 4. CONSTRUCCIÓN DEL MENSAJE

subject = f"FALLA CRÍTICA - BLOQUEO ETL Credit Risk ({datetime.now().strftime('%Y-%m-%d %H:%M')})"
cuerpo_error = ""
detalle = dbutils.widgets.get("lista_detalle")  # Obtener JSON de fallas

try:
    fallas = json.loads(detalle)
    
    if fallas:
        # Se construye el mensaje en una lista para mayor eficiencia y legibilidad.
        partes_error = ["BLOQUEO POR CALIDAD DE DATOS (BRONZE)\\n\\nLas tareas Silver y Gold NO se ejecutaron.\\n\\nFallas detectadas:\\n\\n"]
        for falla in fallas:
            partes_error.append(f" - X REGLA: {falla.get('regla', 'N/A')} ({falla.get('nivel', 'N/A')})\\n")
            partes_error.append(f"   - Columna: {falla.get('columna', 'N/A')}\\n")
            partes_error.append(f"   - Detalle: {falla.get('detalle', 'N/A')}\\n\\n")
        cuerpo_error = "".join(partes_error)
    else:
        cuerpo_error = "Falla Crítica sin detalle: El pipeline se detuvo, pero la información de las reglas fallidas no fue registrada."

except json.JSONDecodeError:
    cuerpo_error = "Falla Crítica: Error al procesar el JSON con el detalle de las fallas. Revisar logs."

mensaje_final = (
    f"Estimado equipo,\\n\\n"
    f"{cuerpo_error}"
    f"Acción Requerida: Revisar los datos fuente de la capa Bronze inmediatamente."
)

In [0]:
# 5. ENVÍO DEL CORREO

msg = EmailMessage()
msg.set_content(mensaje_final)
msg["Subject"] = subject
msg["From"] = sender_email
msg["To"] = dbutils.widgets.get("send_to")  # Uso directo del widget

try:
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, clave_app)
        server.send_message(msg)
    print("Correo de ERROR de Calidad y Bloqueo enviado.")
except Exception as e:
    print(f"Error al enviar el correo: {e}")